<a href="https://colab.research.google.com/github/XGHouFTW/py-cryptopredict/blob/main/Merging_Dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
basepath = "/content/drive/MyDrive/ISEF/Datasets/Stonks/Preprocessed/CryptoStockPrice.csv"
dfbase = pd.read_csv(basepath)

medium = {"news":"/content/drive/MyDrive/ISEF/Datasets/Google News/","reddit":"/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/"}
keyword = ["bitcoin","ethereum","dogecoin","cryptocurrency","economy","finance","politics","pandemic"]
scorelist = {} #dictionary of scorecolumnname and path
for med in medium:
  for word in keyword:
    scorelist[med + "-" + word] = medium[med] + med + "-" + word + "-scored.csv"

In [ ]:
transMonth = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}

a = "2020-01-31	" #from historical prices
def trans(strdate):
    return datetime.datetime(int(strdate[0:4]), int(strdate[5:7]), int(strdate[8:10]))

In [ ]:
start = trans("2020-01-31") #to be compared to new dates for indexing
for i in range(len(dfbase)):
  dfbase.loc[i,"Date"] = trans(dfbase.loc[i,"Date"])

In [ ]:
for scoreset in scorelist:
  dfAdd = pd.read_csv(scorelist[scoreset])

  #initialize new columns with default value 0
  dfbase[scoreset + "PosScoresum"] = 0
  dfbase[scoreset + "PosScoreaverage"] = 0
  dfbase[scoreset + "NegScoresum"] = 0
  dfbase[scoreset + "NegScoreaverage"] = 0
  dfbase[scoreset + "ComScoresum"] = 0
  dfbase[scoreset + "ComScoreaverage"] = 0
  dfbase[scoreset + "Scorecount"] = 0

  for i in range(len(dfAdd)):
    ind = (trans(dfAdd.loc[i,"Date"])-start).days #days since start of timeframe
    if ind < 0 or ind > 731: #remove extraneous entries in dataset
      print("Date out of range error:" + scoreset + str(i) + dfAdd.loc[i,"Date"])
    else:
      dfbase.loc[ind, scoreset + "PosScoresum"] = dfbase.loc[ind, scoreset + "PosScoresum"] + dfAdd.loc[i,"Pos"] 
      dfbase.loc[ind, scoreset + "NegScoresum"] = dfbase.loc[ind, scoreset + "NegScoresum"] + dfAdd.loc[i,"Neg"] 
      dfbase.loc[ind, scoreset + "ComScoresum"] = dfbase.loc[ind, scoreset + "ComScoresum"] + dfAdd.loc[i,"Com"] 
      dfbase.loc[ind, scoreset + "Scorecount"] = dfbase.loc[ind, scoreset + "Scorecount"] + 1
  
  for i in range(len(dfbase)): #compute averages, if entry exists on this date
    if not dfbase.loc[i,scoreset + "Scorecount"] == 0:
      dfbase[scoreset + "PosScoreaverage"] = dfbase[scoreset + "PosScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "NegScoreaverage"] = dfbase[scoreset + "NegScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "ComScoreaverage"] = dfbase[scoreset + "ComScoresum"]/dfbase[scoreset + "Scorecount"]


In [ ]:
dfbase.to_csv("/content/drive/MyDrive/ISEF/Datasets/MergedPriceData.csv", index=False)